In [2]:
from pysat.solvers import Glucose3
from pysat.formula import CNF

from pysat.solvers import Glucose3
from pysat.formula import CNF

def sudoku_solver(grid):
    n = len(grid)
    m = int(n**0.5)

    # Create a SAT solver instance
    solver = Glucose3()

    # Create CNF formula
    cnf_formula = CNF()

    # Variables representing cell values
    variable_indices = {}
    for i in range(1, n+1):
        for j in range(1, n+1):
            for k in range(1, n+1):
                variable = f"{i}_{j}_{k}"
                variable_indices[variable] = len(variable_indices) + 1

    # Constraint 1: Each cell must have exactly one value
    for i in range(1, n+1):
        for j in range(1, n+1):
            cell_variables = [variable_indices[f"{i}_{j}_{k}"] for k in range(1, n+1)]
            cnf_formula.append(cell_variables)
            for k in range(n):
                for l in range(k + 1, n):
                    cnf_formula.append([-cell_variables[k], -cell_variables[l]])

    # Constraint 2: Each value must appear exactly once in each row
    for i in range(1, n+1):
        for k in range(1, n+1):
            row_variables = [variable_indices[f"{i}_{j}_{k}"] for j in range(1, n+1)]
            cnf_formula.append(row_variables)
            for j in range(n):
                for l in range(j + 1, n):
                    cnf_formula.append([-row_variables[j], -row_variables[l]])

    # Constraint 3: Each value must appear exactly once in each column
    for j in range(1, n+1):
        for k in range(1, n+1):
            col_variables = [variable_indices[f"{i}_{j}_{k}"] for i in range(1, n+1)]
            cnf_formula.append(col_variables)
            for i in range(n):
                for l in range(i + 1, n):
                    cnf_formula.append([-col_variables[i], -col_variables[l]])

    # Constraint 4: Each value must appear exactly once in each subgrid
    for x in range(0, n, m):
        for y in range(0, n, m):
            for k in range(1, n+1):
                subgrid_variables = [variable_indices[f"{i+1}_{j+1}_{k}"] for i in range(x, x+m) for j in range(y, y+m)]
                cnf_formula.append(subgrid_variables)
                for p in range(m):
                    for q in range(m):
                        for r in range(q + 1, m):
                            cnf_formula.append([-subgrid_variables[p*m+q], -subgrid_variables[p*m+r]])

    # Add CNF formula to the solver
    solver.append_formula(cnf_formula.clauses)

    # Assign known values
    for i in range(n):
        for j in range(n):
            value = grid[i][j]
            if value != 0:
                solver.add_clause([variable_indices[f"{i+1}_{j+1}_{value}"]])

    # Solve the SAT problem
    if solver.solve():
        # Get the solution
        solution = solver.get_model()

        # Extract the values assigned to each cell
        for i in range(1, n+1):
            row = []
            for j in range(1, n+1):
                for k in range(1, n+1):
                    if variable_indices[f"{i}_{j}_{k}"] in solution:
                        row.append(k)
            print(row)
    else:
        print("No solution found.")

# Example Sudoku grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Solve the Sudoku puzzle
sudoku_solver(sudoku_grid)


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
